
GROUP MEMBERS:

Asad Karim -- 226063

Shweta Pandey -- 225964

In [0]:
import tensorflow as tf
import numpy as np
import os

In [2]:
os.getcwd()

'/content'

In [4]:
!python prepare_data.py shakespeare.txt skp

2020-05-26 06:38:13.786392: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 5604 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900 

In [5]:
from prepare_data import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{' ': 1, 'F': 2, 'I': 3, '\n': 4, 'u': 5, 'q': 6, 'x': 7, 'Q': 8, 'b': 9, 'P': 10, 't': 11, 'o': 12, 'a': 13, 'V': 14, ';': 15, 'E': 16, 'D': 17, '-': 18, '.': 19, 'r': 20, 'p': 21, 'l': 22, 'R': 23, 'N': 24, 'm': 25, 'T': 26, '!': 27, 'g': 28, 'C': 29, 'Y': 30, 'Z': 31, "'": 32, '&': 33, 'z': 34, ',': 35, 'L': 36, '$': 37, '3': 38, 'K': 39, 'j': 40, 'O': 41, 'M': 42, 'y': 43, 'w': 44, 'n': 45, 'J': 46, 'W': 47, 'i': 48, 'A': 49, '?': 50, 's': 51, 'f': 52, 'k': 53, 'S': 54, 'H': 55, 'G': 56, 'v': 57, 'h': 58, ':': 59, 'U': 60, 'e': 61, 'B': 62, 'd': 63, 'X': 64, 'c': 65, '<S>': 0}
66


In [6]:
tf.one_hot(next(iter(data.batch(128))),vocab_size)[127][199]

<tf.Tensor: shape=(66,), dtype=float32, numpy=
array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [0]:
input_dim = vocab_size

hidden_dim = 512

output_dim = vocab_size

batch_size = 128

range = 0.2

W_ih = tf.Variable(tf.random.uniform([input_dim, hidden_dim], minval = -range, maxval = range), dtype=np.float32)

W_hh = tf.Variable(tf.random.uniform([hidden_dim, hidden_dim], minval = -range, maxval = range), dtype=np.float32)

b_h = tf.Variable(tf.random.uniform([hidden_dim], minval = -range, maxval = range), dtype=np.float32)

W_ho = tf.Variable(tf.random.uniform([hidden_dim, output_dim], minval = -range, maxval = range), dtype=np.float32)

b_o = tf.Variable(tf.random.uniform([output_dim], minval = -range, maxval = range), dtype=np.float32)

h_n = tf.reshape(tf.Variable(np.zeros(hidden_dim), dtype= np.float32),(1,512))

def rnn(input, h_n):
  input = tf.reshape(input,(1,66))
  a = tf.matmul(input, W_ih) + b_h + tf.matmul(h_n, W_hh)
  h_n = tf.reshape(tf.nn.sigmoid(a), (1,512))
  o = b_o + tf.matmul(h_n, W_ho)
  return h_n, o





In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [0]:
epochs = tf.Variable(20)
n_time_steps = tf.Variable(128)
seq_size = tf.Variable(200)
e = tf.Variable(1)
def train():
  for epoch in tf.range(epochs):
    #print(epoch)
    print('Start of epoch %d', (epoch,))

    for time_step in tf.range(n_time_steps):
      #print(time_step)
      char_n = tf.one_hot(next(iter(data.batch(128))),vocab_size)
      with tf.GradientTape() as tape:
        hidden_states = tf.TensorArray(tf.float32,size=0,dynamic_size=True, element_shape = (1,hidden_dim))
        outputs = tf.TensorArray(tf.float32,size=0,dynamic_size=True)
        loss = tf.TensorArray(tf.float32,size=0,dynamic_size=True)
        for t in tf.range(seq_size-1):
          if tf.less(t, 1):
            h_n = tf.reshape(tf.Variable(np.zeros(hidden_dim), dtype= np.float32),(1,512))
          else:
            h_n = hidden_states.read(t-1)
          char_n_tmp = char_n[time_step][t]
          hidden_state, output = rnn(char_n_tmp, h_n)
          outputs.write(outputs.size(),output).mark_used()
          hidden_states.write(t, hidden_state).mark_used()
          #print("output", output)
          #print("labels", char_n[time_step][t+1])
          loss.write(t, tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=char_n[time_step][t+1]))).mark_used()
        # time_ste.mark_used()
        loss_value = loss.stack()
        grads = tape.gradient(loss_value,[W_hh, W_ho, W_ih, b_h, b_o])
        #print(zip(grads, [W_hh, W_ho, W_ih, b_h, b_o]))
        optimizer.apply_gradients(zip(grads, [W_hh, W_ho, W_ih, b_h, b_o]))


      if time_step % 200 == 0:
        print('Training loss (for one batch) at step %s: %s' % (time_step, float(loss_value[-1])))
        print('Seen so far: %s samples' % ((time_step + 1) * 128))



In [28]:
train()

Start of epoch %d (<tf.Tensor: shape=(), dtype=int32, numpy=0>,)
Training loss (for one batch) at step tf.Tensor(0, shape=(), dtype=int32): 0.6403665542602539
Seen so far: tf.Tensor(128, shape=(), dtype=int32) samples
Start of epoch %d (<tf.Tensor: shape=(), dtype=int32, numpy=1>,)
Training loss (for one batch) at step tf.Tensor(0, shape=(), dtype=int32): 0.3942781686782837
Seen so far: tf.Tensor(128, shape=(), dtype=int32) samples
Start of epoch %d (<tf.Tensor: shape=(), dtype=int32, numpy=2>,)
Training loss (for one batch) at step tf.Tensor(0, shape=(), dtype=int32): 0.39575669169425964
Seen so far: tf.Tensor(128, shape=(), dtype=int32) samples
Start of epoch %d (<tf.Tensor: shape=(), dtype=int32, numpy=3>,)
Training loss (for one batch) at step tf.Tensor(0, shape=(), dtype=int32): 0.4170176088809967
Seen so far: tf.Tensor(128, shape=(), dtype=int32) samples
Start of epoch %d (<tf.Tensor: shape=(), dtype=int32, numpy=4>,)
Training loss (for one batch) at step tf.Tensor(0, shape=(), d

In [0]:
def rnn_extended(input, h_n):
  input = tf.reshape(input,(1,66))
  a = tf.matmul(input, W_ih) + b_h + tf.matmul(h_n, W_hh)
  h_n = tf.reshape(tf.nn.sigmoid(a), (1,512))
  o = tf.nn.softmax(b_o + tf.matmul(h_n, W_ho))
  return h_n, o

In [0]:
char_nn = tf.convert_to_tensor(
    ([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))
h_nn = tf.reshape(tf.Variable(np.zeros(hidden_dim), dtype= np.float32),(1,512))

def generate_text(char, h): 
  hidden, out = rnn_extended(char,h)
  out = tf.reshape(out, (66))
  index = tf.math.argmax(out, axis=0)
  out = tf.one_hot(index, vocab_size)
  #print(int(index))
  return ind_to_ch.get(int(index)), hidden, out


In [84]:
txt_len = 1000
words = ''
for t in tf.range(txt_len):
  if t == 0:
    txt, hidden, out = generate_text(char_nn,h_nn)
  else: 
    txt, hidden, out = generate_text(out, hidden)
  words = words + txt


print(words)


 the sender the senst the sers and the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s ard the s

The text generated looks like its stuck in a loop. A local minima



